#**Deploy COVID-19 Prediction Model into Azure ML Workspce**


In [23]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/COVID19/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/COVID19


In [24]:
from platform import python_version
print(python_version())

3.6.9


In [25]:
!pip install --upgrade azureml


Requirement already up-to-date: azureml in /usr/local/lib/python3.6/dist-packages (0.2.7)


In [26]:
!pip install --upgrade azureml.core

  Using cached https://files.pythonhosted.org/packages/d9/f2/ba5966efa8d6b778f0dab994152b5cd7a2995c0b0194fdde8100b75622e0/azureml_core-1.6.0-py3-none-any.whl


In [0]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model

In [0]:
import azureml.core
from azureml.core import Workspace

In [29]:
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.6.0


In [0]:
##by me
# from azureml.core.webservice import Webservice
# from azureml.core.image import ContainerImage
# #from azureml.core.webservice import Aciwebservice
# from azureml.core.conda_dependencies import CondaDependencies

In [0]:
#!pip install azure-cli

#**1. Get Workspace Information**

---



---



---



In [0]:
# from azureml.core.authentication import AzureCliAuthentication

# cli_auth = AzureCliAuthentication()

# ws = Workspace(subscription_id="my-subscription-id",
#                resource_group="my-ml-rg",
#                workspace_name="my-ml-workspace",
#                auth=cli_auth)

# print("Found workspace {} at location {}".format(ws.name, ws.location))

In [30]:
azureml.core.workspace.Workspace

azureml.core.workspace.Workspace

In [31]:
from azureml.core.authentication import InteractiveLoginAuthentication


interactive_auth = InteractiveLoginAuthentication(tenant_id="872b50a3-f641-4a7d-b1c4-cce3c3da9318")
 

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code G2L299J4N to authenticate.
Interactive authentication successfully completed.


In [0]:
# from azureml.core import Workspace
# ws = Workspace.create(name='Covid_predictor_ws',
#                          subscription_id='413dc411-db88-4292-b3a0-4bbf10eff75e',
#                          resource_group='Covid_predictor_rg',
#                          create_resource_group=True,
#                          location='southcentralus'
#                         )

In [0]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="872b50a3-f641-4a7d-b1c4-cce3c3da9318")

ws = Workspace(subscription_id="413dc411-db88-4292-b3a0-4bbf10eff75e",
               resource_group="Covid_predictor_rg",
               workspace_name="Covid_predictor_ws",
               auth=interactive_auth)

In [34]:
print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace Covid_predictor_ws at location southcentralus


#**2.Register the model**

In [0]:
 from azureml.core.model import Model
# model = Model(ws, 'my_model_name')

In [38]:
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "xception_CovidVsNormal_Final_Best_model_weights.h5",
                        model_name = "Covid-19_Prediction",
                        description = "COVID19 Prediction using xception model",
                        workspace = ws)

Registering model Covid-19_Prediction


413dc411-db88-4292-b3a0-4bbf10eff75e
413dc411-db88-4292-b3a0-4bbf10eff75e

#**3.Define inference configuration**

In [0]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("keras")
conda_dep.add_conda_package("opencv")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="modelscore.py",
                                   environment=myenv)

#**4.Deploy The Model**

In [0]:
#from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
from azureml.core.webservice import AciWebservice,Webservice

In [0]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

In [0]:
model = Model(ws, name='Covid-19_Prediction')

In [0]:
# service = Model.deploy(ws, 'covidwebservice', [model], inference_config, deployment_config)

# service.wait_for_deployment(True)



Running..................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [46]:
#print(service.state)
print("scoring URI: " + service.scoring_uri)

scoring URI: http://40c5e72a-fcfd-4b3d-b075-0e6b552a2d18.southcentralus.azurecontainer.io/score


In [45]:
print(service.state)

Healthy


###**Update The Model**


In [0]:
service_name = 'covidwebservice'
# Retrieve existing service.
service = Webservice(name=service_name, workspace=ws)

In [0]:
# Update to new model(s).
service.update(models=[model], inference_config=inference_config)
print(service.state)
print(service.get_logs())

Healthy
2020-05-31T03:02:43,441568578+00:00 - iot-server/run 
2020-05-31T03:02:43,442851787+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-05-31T03:02:43,456657989+00:00 - gunicorn/run 
2020-05-31T03:02:43,457007191+00:00 - rsyslog/run 


In [0]:
service.auth_enabled

False

In [0]:
print(service.swagger_uri)

http://40c5e72a-fcfd-4b3d-b075-0e6b552a2d18.southcentralus.azurecontainer.io/swagger.json


In [0]:
primary, secondary = service.get_keys()
print(primary)

In [0]:
deployment_config

In [0]:
# Get the deployed service
service = AciWebservice(name='covidwebservice', workspace=ws)

# Check its state
print(service.state)

Healthy


In [0]:
service.get_logs()

'2020-05-31T03:11:03,847297091+00:00 - iot-server/run \n2020-05-31T03:11:03,850313913+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_6848fd76935549cdd162e1b5a4c8aa4e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-05-31T03:11:03,860257986+00:00 - rsyslog/run \n2020-05-31T03:11:03,861986398+00:00 - gunicorn/run 

###**Request-response consumption**

In [0]:
import requests
import json
import base64

In [0]:
# filePath=r"normal.jpeg"

In [0]:
filePath=r"test1.jpeg"

In [0]:
with open(filePath, "rb") as image:
    f = image.read()
    b = bytearray(f)
    str = base64.b64encode(b)
    str=str.decode("utf-8")
    json_data={'data':str}
    json_data= json.dumps(json_data)
    jsonData=json.loads(json_data)
    test_samples = bytes(json_data, encoding='utf8')

In [60]:
# predict using the deployed model
result = service.run(input_data=test_samples)
print(result)

{'label': 'COVID-19', 'prediction': '0.99993455', 'elapsed_time': 0.5120422840118408}


In [61]:
import requests
import json

scoring_uri = 'http://40c5e72a-fcfd-4b3d-b075-0e6b552a2d18.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

# predict using the deployed model
# result = service.run(input_data=test_samples)

response = requests.post(scoring_uri, data=json_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

200
0:00:00.908430
{'label': 'COVID-19', 'prediction': '0.99993455', 'elapsed_time': 0.6153292655944824}
